In [1]:
import random
import numpy as np
import csv
import re
import os 
import networkx as nx
from networkx.algorithms import approximation
from parse import read_input_file, write_output_file
from utils import is_valid_network, average_pairwise_distance
from pulp import *
import matplotlib.pyplot as plt
import time 
import random as rand

In [2]:
def sol1(G): 
    weighted_set = approximation.min_weighted_dominating_set(G)
    # need a way to connect these vertices cheapest way possible 
    all_potential_T = []
    def helper(G, T, vertices):
        nonlocal all_potential_T
        if all([a in T.nodes() for a in vertices]): 
            all_potential_T.append(T)      
        for edge in G.edges():
            if edge not in T.edges():
                if ((edge[0] not in T.nodes() and edge[1] in T.nodes()) or (edge[0] in T.nodes() and edge[1] not in T.nodes())):
                    new_T = T.copy()
                    new_T.edges()
                    new_T.add_edge(edge[0], edge[1], weight = G[edge[0]][edge[1]]["weight"])
                    if not any([nx.is_isomorphic(new_T, tree) for tree in all_potential_T]):
                        helper(G, new_T, vertices)
                        
    first_T = nx.Graph()
    first_T.add_node(weighted_set.pop())
    helper(G, first_T, weighted_set)
    return min(all_potential_T, key = lambda x: average_pairwise_distance(x))


def add_weighted_path(path, T, G): 
    for i in range(len(path)-1):
        u = path[i]
        v = path[i+1]
        if not ((u,v) in T.edges() and (v,u) in T.edges()): 
            T.add_edge(u, v, weight = G[u][v]["weight"])
def average_cost(path, G): 
    total = 0 
    for i in range(len(path) - 1):
        total += G[path[i]][path[i+1]]["weight"]
    return total


    
# find the cheapest way to connect the weighted_set together     
def sol25(G, x):
    count = 0
    start = time.time() 
    last = 0 
    all_potential_T = []
    for i in range(x):     
        connectivity = {} 
        for a in G.nodes():
            neighbor_size = len(list(G.neighbors(a))) 
            if neighbor_size== len(G.nodes()) - 1: 
                T = nx.Graph() 
                T.add_node(a)
                return (T, 1)  
            connectivity[a] = neighbor_size
        weighted_set = approximation.min_weighted_dominating_set(G)
        top_connect = sorted(connectivity.keys(), key = lambda x: connectivity[x], reverse = False)
        weighted_set = list(weighted_set)
        while len(weighted_set) < 10 and (len(top_connect) != 0):
            a = top_connect.pop()
            b = rand.randint(0, len(G.nodes())-1) 
            if a not in weighted_set and b not in weighted_set: 
                weighted_set.append(a)
                weighted_set.append(b)
        def helper(G, T, vertexes):
            nonlocal count 
            nonlocal all_potential_T
            nonlocal last
            nonlocal start 
            if (time.time() - start) > 180: 
                return None
            count += 1
#             if count % 100 == 0:
#                 print("Length of list at time" + str(time.time()- start) + " second is " + str(len(all_potential_T)))
            node_set = set(list(T.nodes()))
            for a in T.nodes():
                node_set.update(set(list(G.neighbors(a))))
            if len(node_set) == len(G.nodes()):
                trimmer(T) 
            elif len(all_potential_T) - last < 3000:
                for u in vertexes: 
                    if u not in T.nodes():
                        try: 
                            random_v = random.sample(T.nodes(),min([15, len(T.nodes())]))
                            G_simp = G.subgraph(random_v + [a for a in G.nodes() if a not in T.nodes()])
                            #small = min([i for i nx.single_source_shortest_path_length(G_simp, u).values() if i != 0])
                            shortest_path = nx.single_target_shortest_path(G_simp, u )
                            added_path = min([i for i in shortest_path.values() if i[0] in random_v], key = lambda x: average_cost(x, G))
                            new_T = T.copy()
                            add_weighted_path(added_path, new_T, G)
                            if nx.is_tree(new_T) and not any([nx.is_isomorphic(new_T, tree) for tree in all_potential_T]):
                                helper(G, new_T, vertexes)
                        except Exception as e: 
                            a = 1
            else: 
                last = len(all_potential_T) 
    #                     for v in random.sample(T.nodes(),min([15, len(T.nodes())])):
    #                         new_T = T.copy()
    #                         try:
    #                             G_simp = G.subgraph([v] + [a for a in G.nodes() if a not in T.nodes()])
    #                             #added_path = nx.dijkstra_path(G.subgraph([v] + [a for a in G.nodes() if a not in T.nodes()]), source = u, target = v)
    #                             added_path = min(nx.all_simple_paths(G_simp, u, v, cutoff = 1.5* len(next(nx.shortest_simple_paths(G_simp, u, v)))), key = lambda x: average_cost(x, G_simp))
    #                             add_weighted_path(added_path, new_T, G)
    #                             if nx.is_tree(new_T) and not any([nx.is_isomorphic(new_T, tree) for tree in all_potential_T]) :
    #                                 helper(G, new_T, vertexes)
    #                         except:
    #                             a = 1
    #                             #print("Couldn't find appropriate path")

        def trimmer(T):
            og_T = T.copy()
            nonlocal all_potential_T
            for edge in T.edges():
                new_T = T.copy() 
                if len(list(og_T.neighbors(edge[0]))) == 1: 
                    new_T.remove_edge(edge[0], edge[1])
                    new_T.remove_node(edge[0])
                elif len(list(og_T.neighbors(edge[1]))) == 1: 
                    new_T.remove_edge(edge[0], edge[1])
                    new_T.remove_node(edge[1])
                node_set = set(list(new_T.nodes()))
                for a in new_T.nodes():
                    node_set.update(set(list(G.neighbors(a))))
                if len(node_set) == len(G.nodes()) and average_pairwise_distance(new_T) < average_pairwise_distance(og_T):
                    all_potential_T.append(new_T)
                    T = new_T
            all_potential_T.append(T)
            
        shortest_path = [nx.dijkstra_path(G, source = i, target = j ) for (i, j) in ((i, j) for i in weighted_set for j in weighted_set if i < j)]
        for path in shortest_path:
            T = nx.Graph()
            add_weighted_path(path, T, G)
            helper(G, T, weighted_set)
        trimmer(nx.minimum_spanning_tree(G))
    return (min(all_potential_T, key = lambda x: average_pairwise_distance(x)), len(all_potential_T))


def trimmer_back(T): 
        #nonlocal all_potential_T
        for edge in T.edges():
            new_T = T.copy() 
            new_T.remove_edge(edge[0], edge[1])
            if new_T.neighbors(edge[0]) == 1: 
                new_T.remove_node(edge[0])
            else: 
                new_T.remove_node(edge[1])
            node_set = set(list(new_T.nodes()))
            for a in new_T.nodes():
                node_set.update(set(list(G.neighbors(a))))
            if len(node_set) == len(G.nodes()):
                all_potential_T.append(T) 
                trimmer(T)

def sol3(G):
    #first_tree, size = sol25(G, 1)
    #second_tree, size_2 = sol25(G, 1)
    #num = int(np.round(1 + 2*np.log(np.mean(size + size_2))))
    return sol25(G, 10)[0]


def sol4(G):
    start = time.time() 
    all_potential_T = [] 
    def trimmer(T):
        og_T = T.copy()
        nonlocal all_potential_T
        for edge in T.edges():
            new_T = T.copy() 
            if len(list(og_T.neighbors(edge[0]))) == 1: 
                new_T.remove_edge(edge[0], edge[1])
                new_T.remove_node(edge[0])
            elif len(list(og_T.neighbors(edge[1]))) == 1: 
                new_T.remove_edge(edge[0], edge[1])
                new_T.remove_node(edge[1])
            node_set = set(list(new_T.nodes()))
            for a in new_T.nodes():
                node_set.update(set(list(G.neighbors(a))))
            if len(node_set) == len(G.nodes()) and average_pairwise_distance(new_T) < average_pairwise_distance(og_T):
                all_potential_T.append(new_T)
                T = new_T
        all_potential_T.append(T)
        

    def helper_func(G):
        nonlocal start 
        nonlocal all_potential_T 
#         if (time.time() - start) % 30 < .5 and (time.time() - start) % 30 > 0: 
#             print("After" + str(time.time() - start) + " seconds we have + " + str(len(all_potential_T)) + " trees")
        if (time.time() - start) > 240: 
            return None
        
#         if len(all_potential_T) % 100 == 0: 
#             print(len(all_potential_T))
        useless_edge = [] 
        for edge in G.edges(): 
            if len(list(G.neighbors(edge[0]))) > 1 and len(list(G.neighbors(edge[1]))) >  1:
                try: 
                    b = iter(nx.all_simple_paths(G, edge[0], edge[1]))
                    next(b)
                    next(b) 
                    useless_edge.append(edge) 
                except: 
                    a = "Doesn't have edge" 
        if len(useless_edge) != 0: 
            useless_edge = sorted(useless_edge, key = lambda x: G[x[0]][x[1]]["weight"])
            useless_edge_set = set(useless_edge[:4])
            useless_edge_set.update(set(useless_edge[:-4]))
            useless_edge_set.update(set(rand.sample(useless_edge, min(len(useless_edge), 4))))
            for edge in list(useless_edge): 
                new_G = G.copy() 
                new_G.remove_edge(edge[0], edge[1])
                if len(list(new_G.neighbors(edge[0]))) == 1: 
                    new_G.remove_node(edge[0])
                elif len(list(new_G.neighbors(edge[1]))) == 1: 
                    new_G.remove_node(edge[1])
                if nx.is_connected(new_G): 
                    spanning_tree = nx.minimum_spanning_tree(new_G)
                    if not any([nx.is_isomorphic(spanning_tree, tree) for tree in all_potential_T]) and len(all_potential_T) < 1000:
                        all_potential_T.append(spanning_tree)
                        trimmer(spanning_tree)
                        helper_func(new_G)
    helper_func(G)
    trimmer(nx.minimum_spanning_tree(G))
    return min(all_potential_T, key = lambda x: average_pairwise_distance(x))
                
def sol5(G): 
    ratio = len(G.edges())/len(G.nodes())
    print(ratio)
    if ratio > 2.75: 
        return sol3(G)
    else: 
        return sol4(G)
    
    

In [3]:
G = read_input_file("10.in")

In [4]:
start = time.time() 
ans = sol5(G)
print(time.time() - start)
average_pairwise_distance(ans)

1.8
7.66766095161438


28.686666666666667

In [8]:
# for file in os.listdir("inputs")

G25 = read_input_file("25.in")
start  = time.time()
ans25 = sol5(G25)
print(time.time() - start)
G50 = read_input_file("50.in")
start  = time.time()
ans50 = sol5(G50)
print(time.time() - start)
G100 = read_input_file("100.in")
start  = time.time()
ans100 = sol5(G100)
print(time.time() - start)

5.2
84.62483620643616
17.96
18.408946990966797
46.42
0.00020503997802734375


In [9]:
average_pairwise_distance(ans25)

1.93

In [10]:
def draw_graph(G): 
    pos=nx.spring_layout(G)
    plt.figure(figsize = (20, 10))
    nx.draw(G, pos)
    labels = nx.get_edge_attributes(G,'weight')
    nx.draw_networkx(G, pos,  with_labels= True)
    nx.draw_networkx_edge_labels(G,pos,edge_labels=labels, );


In [11]:
failed_graphs = []
completed_graphs = [] 

In [12]:
for graph in sorted(os.listdir("inputs"), reverse = True):
    try: 
        G = read_input_file("inputs/" + graph)
    except:
        G = 0
        failed_graphs.append(graph)
        
    if G != 0 and graph not in completed_graphs: 
        print(graph + " has " + str(len(G.nodes())) + " nodes " + str(len(G.edges)) + " edges.")
        start = time.time() 
        T = sol5(G)
        print("Graph: " + graph +  " took " + str(time.time() - start) + " seconds. This graph had: " + str(len(G.nodes())) + " nodes")
        print("Saving graph with average pairwise distance " + str(average_pairwise_distance(T)) + " to " + 'outputs/' + graph[:-2] + 'out')
        write_output_file(T, 'outputs/' + graph[:-2] + 'out')
        completed_graphs.append(graph) 

small-99.in has 25 nodes 64 edges.
2.56
Graph: small-99.in took 275.47936511039734 seconds. This graph had: 25 nodes
Saving graph with average pairwise distance 46.614948717948714 to outputs/small-99.out
small-98.in has 25 nodes 232 edges.
9.28
Graph: small-98.in took 9.483349084854126 seconds. This graph had: 25 nodes
Saving graph with average pairwise distance 0.438 to outputs/small-98.out
small-97.in has 14 nodes 19 edges.
1.3571428571428572
Graph: small-97.in took 10.026448726654053 seconds. This graph had: 14 nodes
Saving graph with average pairwise distance 80.75 to outputs/small-97.out
small-96.in has 24 nodes 23 edges.
0.9583333333333334
Graph: small-96.in took 0.041493892669677734 seconds. This graph had: 24 nodes
Saving graph with average pairwise distance 2.769230769230769 to outputs/small-96.out
small-95.in has 25 nodes 75 edges.
3.0
Graph: small-95.in took 181.08178305625916 seconds. This graph had: 25 nodes
Saving graph with average pairwise distance 44.200607142857145 to

Graph: small-45.in took 7.572418928146362 seconds. This graph had: 25 nodes
Saving graph with average pairwise distance 21.487179487179485 to outputs/small-45.out
small-44.in has 25 nodes 25 edges.
1.0
Graph: small-44.in took 0.1907041072845459 seconds. This graph had: 25 nodes
Saving graph with average pairwise distance 7.66794871794872 to outputs/small-44.out
small-43.in has 25 nodes 46 edges.
1.84
Graph: small-43.in took 23.78227138519287 seconds. This graph had: 25 nodes
Saving graph with average pairwise distance 10.470760233918128 to outputs/small-43.out
small-41.in has 25 nodes 71 edges.
2.84
Graph: small-41.in took 183.02327609062195 seconds. This graph had: 25 nodes
Saving graph with average pairwise distance 44.67805555555555 to outputs/small-41.out
small-39.in has 25 nodes 55 edges.
2.2
Graph: small-39.in took 326.61181712150574 seconds. This graph had: 25 nodes
Saving graph with average pairwise distance 65.85351666666666 to outputs/small-39.out
small-38.in has 25 nodes 78 

KeyboardInterrupt: 

In [10]:
sorted(os.listdir("inputs"), reverse = True)

['small-99.in',
 'small-98.in',
 'small-97.in',
 'small-96.in',
 'small-95.in',
 'small-94.in',
 'small-93.in',
 'small-92.in',
 'small-91.in',
 'small-90.in',
 'small-9.in',
 'small-89.in',
 'small-88.in',
 'small-87.in',
 'small-86.in',
 'small-85.in',
 'small-84.in',
 'small-83.in',
 'small-82.in',
 'small-81.in',
 'small-80.in',
 'small-8.in',
 'small-79.in',
 'small-78.in',
 'small-77.in',
 'small-76.in',
 'small-75.in',
 'small-74.in',
 'small-73.in',
 'small-72.in',
 'small-71.in',
 'small-70.in',
 'small-7.in',
 'small-69.in',
 'small-68.in',
 'small-67.in',
 'small-66.in',
 'small-65.in',
 'small-64.in',
 'small-63.in',
 'small-62.in',
 'small-61.in',
 'small-60.in',
 'small-6.in',
 'small-59.in',
 'small-58.in',
 'small-57.in',
 'small-56.in',
 'small-55.in',
 'small-54.in',
 'small-53.in',
 'small-52.in',
 'small-51.in',
 'small-50.in',
 'small-5.in',
 'small-49.in',
 'small-48.in',
 'small-47.in',
 'small-46.in',
 'small-45.in',
 'small-44.in',
 'small-43.in',
 'small-42.in

In [ ]:
start = time.time()
G = read_input_file("inputs/medium-6.in")
len(G.nodes()), len(G.edges())
solu = sol4(G)
print(time.time() - start)

In [ ]:
def average_cost(path, G): 
    total = 0 
    for i in range(len(path) - 1):
        total += G[path[i]][path[i+1]]["weight"]
    return total

min(nx.all_simple_paths(G, 0, 1, cutoff = 3* len(next(nx.shortest_simple_paths(G, 0, 1)))), key = lambda x: average_cost(x, G))

In [ ]:
next(nx.shortest_simple_paths(G, 0, 1))


In [ ]:
nx.is_connected()

In [ ]:
G[6][3]

In [ ]:
G.edges()

In [ ]:
G.remove_edge(0, 21)

In [ ]:
G.edges()

In [ ]:
T = G.copy()
nx.is_isomorphic(T, G)

In [ ]:

draw_graph(G)

In [ ]:
any([False, False, False])

In [ ]:
G

In [ ]:
i = iter(nx.all_simple_paths(G, 0, 1))

In [ ]:
b = iter([1, 2])

In [ ]:
next(b)